In [7]:
import open3d
import numpy as np
import pandas as pd
import os
import tqdm
import copy

import matplotlib.pyplot as plt
import utils.registration as registration
import utils.functions as functions
import utils.transform as transform
import utils.pointcloud as pointcloud
import utils.fread as fread
import utils.FCGF as FCGF

from utils.config import Config

from scipy.signal import argrelmin, argrelmax

In [9]:
config = Config(
        sequence_dir="data/raw_data",
        feature_dir="data/features",
        output_dir="results/std_values",
        experiment="exp_12",
        trial="trial_1",
        subject="subject-1",
        sequence="01",
        groundtruth_dir="data/trajectories/groundtruth",
    )
    
config.voxel_size=0.03
config.target_fps=20
config.min_std=0.5
config.cutoff_margin=5

In [14]:
def plot_std_cutoffs(config: Config):
    sequence_ts = fread.get_timstamps_from_images(config.get_sequence_dir(), ".depth.png")
    # sequence_ts = fread.sample_timestamps(sequence_ts, config.target_fps)

    # calculate the standard deviation of the depth data
    std_values = []

    for t in range(len(sequence_ts)):
        depth_img_file = os.path.join(config.get_sequence_dir(), f"frame-{sequence_ts[t]}.depth.png")
        std_values.append(registration.calc_std(depth_img_file, 4000))
        
    std_values = np.array(std_values)
        
    cutoffs = registration.get_cutoff_sequence(std_values, config.target_fps, config.min_std, config.threshold, config.cutoff_margin)

    min_indices = argrelmin(std_values, order=config.target_fps // 2)[0]
    min_indices = min_indices[np.where(np.abs(std_values[min_indices] - config.min_std) < config.threshold)[0]]

    plt.figure(figsize=(20, 6))
    plt.plot(std_values)

    plt.scatter(min_indices, std_values[min_indices], c="b", marker="x")

    for x in min_indices:
        plt.axvline(x, c="b", linestyle="--")

    plt.axhline(y=0.5, color="r", linestyle="--")
    plt.ylim(0, 4)
    plt.xlim(0, len(std_values))

    plt.xlabel("Frame #")
    plt.ylabel("Std. of Distances to the camera")
    plt.savefig(config.get_output_file(f"{config.get_file_name()}.jpeg"))
    plt.close()

In [15]:
for trial in os.listdir(os.path.join(config.feature_dir, config.experiment)):
    config.trial = trial
    for subject in os.listdir(os.path.join(config.feature_dir, config.experiment, config.trial, str(config.voxel_size))):
        config.subject = subject    
        for sequence in os.listdir(os.path.join(config.feature_dir, config.experiment, config.trial, str(config.voxel_size), config.subject)):
            config.sequence = sequence
            print(f"Processing: {config.experiment} >> {config.trial} >> {config.subject} >> {config.sequence}")
            plot_std_cutoffs(config)

Processing: exp_12 >> trial_1 >> subject-1 >> 01
Processing: exp_12 >> trial_1 >> subject-1 >> 02
Processing: exp_12 >> trial_1 >> subject-1 >> 03
Processing: exp_12 >> trial_1 >> subject-1 >> 04
Processing: exp_12 >> trial_1 >> subject-1 >> 05
Processing: exp_12 >> trial_2 >> subject-1 >> 01
Processing: exp_12 >> trial_2 >> subject-1 >> 02
Processing: exp_12 >> trial_2 >> subject-1 >> 03
Processing: exp_12 >> trial_2 >> subject-1 >> 04
Processing: exp_12 >> trial_2 >> subject-1 >> 05
Processing: exp_12 >> trial_3 >> subject-1 >> 01
Processing: exp_12 >> trial_3 >> subject-1 >> 02
Processing: exp_12 >> trial_3 >> subject-1 >> 03
Processing: exp_12 >> trial_3 >> subject-1 >> 04
Processing: exp_12 >> trial_3 >> subject-1 >> 05
